## Sensitivity analysis ABM
Univariate sensitivity analysis conducted for the machines, education level, wage worker salary, and fixed costs of rice, maize, shrimp, and coconut farmers. Since the batchrunner is slow due to the datacollector, it is recommended to first look at the required variables, and then select those in the model3.py file in line 230-350. 

In [ ]:
import pandas as pd
import numpy as np
from mesa import batch_run
from Model3 import RiverDeltaModel

# Number of runs per parameter
N = 100

# Set up the parameter values
machine_values = np.linspace(0, 1, N)
education_values = np.linspace(0, 1, N)
ww_salary_values = np.linspace(0, 2, N)
fixed_costs_rice_values = np.linspace(0, 32000000, N)
fixed_costs_maize_values = np.linspace(0, 7000000, N)
fixed_costs_coconut_values = np.linspace(0, 40000000, N)
fixed_costs_shrimp_values = np.linspace(0, 8000000, N)

# Create a function to run the sensitivity analysis
def run_sensitivity(param_name, values):
    # Set base value
    fixed_params = {
        "machines_experiment": 2,
        "education_level": 2,
        "ww_salary_increase":1,
        "fixed_costs_rice" : 15900000,
        "fixed_costs_maize" : 3400000,
        "fixed_costs_coconut": 20000000,
        "fixed_costs_shrimp" : 3900000
    }

    # change the chosen parameter value to the values, while keeping the other values constant
    fixed_params[param_name] = values

    # Run the model with 1 iteration, and save the data at the last step
    result = batch_run(
        RiverDeltaModel,
        parameters=fixed_params,
        iterations=1,              # 1 run per parameter 
        max_steps=300,
        number_processes=8,
        data_collection_period=300,
        display_progress=True,
    )

    # Create a dataframe with the result
    df = pd.DataFrame(result)
    return df



In [ ]:
# Sensitivity for machines
df_machines = run_sensitivity("machines_experiment", machine_values)

# Sensitivity for education_level
df_education = run_sensitivity("education_level", education_values)

# Sensitivity for ww_salary
df_ww_salary = run_sensitivity("ww_salary_increase", ww_salary_values)

# Sensitivity for fixed costs per crop
df_fixed_rice = run_sensitivity("fixed_costs_rice", fixed_costs_rice_values)
df_fixed_maize = run_sensitivity("fixed_costs_maize", fixed_costs_maize_values)
df_fixed_coconut = run_sensitivity("fixed_costs_coconut", fixed_costs_coconut_values)
df_fixed_shrimp = run_sensitivity("fixed_costs_shrimp", fixed_costs_shrimp_values)


### Number of migrations

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm

# Determine baseline values, these are used in the plots as a reference point
baseline_values = {
    "machines_experiment": 0.10,
    "education_level": 0.32,
    "ww_salary_increase": 1,
    "fixed_costs_rice": 15900000,
    "fixed_costs_maize": 3400000,
    "fixed_costs_coconut": 20000000,
    "fixed_costs_shrimp": 3900000
}

def plot_lowess(df, x_col, y_col="Migrated_households", step_col="Step", step_value=300, 
                title=None, xlabel=None, ylabel="Percentage migrated households"):
    
    # Select the last timestep
    df_filtered = df[df[step_col] == step_value]

    # Determine x, y values
    x = df_filtered[x_col]
    y = (df_filtered[y_col] / 275)  # Set start number of households

    # LOWESS smoothing
    lowess = sm.nonparametric.lowess
    smoothed = lowess(y, x, frac=0.2)

    # Create a scatterplot
    plt.figure(figsize=(8, 5))
    plt.scatter(x, y, alpha=0.6, color="green", label="Data")
    plt.plot(smoothed[:, 0], smoothed[:, 1], color="navy", linewidth=2.5, label="LOWESS trend")

    # Verticale baseline for the base values
    if x_col in baseline_values:
        baseline_x = baseline_values[x_col]
        plt.axvline(baseline_x, color="red", linestyle="--", linewidth=2, label="Baseline")

    plt.title(title if title else f"{x_col} vs. {y_col}")
    plt.xlabel(xlabel if xlabel else x_col.replace("_", " ").title())
    plt.ylabel(ylabel)
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.ylim(0.15, 0.55)
    plt.show()


### Plot all variables

In [ ]:
# Create plots for all variables
plot_lowess(df_machines, "machines_experiment", title="Effect of machine use on migration in ABM", xlabel="Percentage people using machines")

plot_lowess(df_education, "education_level", title="Effect of education level on migration in ABM", xlabel="Education Level")

plot_lowess(df_ww_salary, "ww_salary_increase", title="Effect of wage worker salary on migration in ABM", xlabel="Percentage salary received")

plot_lowess(df_fixed_rice, "fixed_costs_rice", title="Effect of rice production costs on migration in ABM", xlabel="Fixed crop costs (vnd)")

plot_lowess(df_fixed_maize, "fixed_costs_maize", title="Effect of maize production costs on migration in ABM", xlabel="Fixed crop costs (vnd)")

plot_lowess(df_fixed_coconut, "fixed_costs_coconut", title="Effect of coconut production costs on migration in ABM", xlabel="Fixed crop costs (vnd)")

plot_lowess(df_fixed_shrimp, "fixed_costs_shrimp", title="Effect of shrimp production costs on migration in ABM", xlabel="Fixed crop costs (vnd)")
